In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib
from skimage import io
from scipy.spatial import distance
import PIL.Image
import PIL.ImageDraw
import time


In [8]:
simple = "ded.png" # изображение для сравнения

In [9]:
def show(img, flag = 0):
    if flag == 1:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots()
    ax.imshow(img)
    fig.set_figwidth(10)
    fig.set_figheight(10) 
    plt.show()

In [10]:
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()

In [11]:
img = io.imread(simple)

In [12]:
dets = detector(img, 1)

RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [ ]:
shape = 0
for k, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(img, d)
    rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    f = cv2.rectangle(rgb_image, (d.left(), d.top()), (d.right(), d.bottom()), (0, 255, 255), 10)
    show(f)

In [ ]:
f = cv2.rectangle(rgb_image, (d.left(), d.top()), (d.right(), d.bottom()), (0, 255, 255), 10)
face_descriptor1 = facerec.compute_face_descriptor(f, shape)

# # Камера

In [5]:
cap = cv2.VideoCapture(-1)
timeToslip = 0
face_descriptor2 = 0
a = 0
while cap.isOpened():
    
    timeToslip +=1
    #Получение с камеры 
    ret, img = cap.read()
    
    # с лицами
    dets = detector(img, 1)
    shape = 0
    for k, d in enumerate(dets):
        shape = sp(img, d)
    #другое цветовое пространство которое не работает TODO PS: Оказывается оно и не должно работать, схема уже RGB
    #rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #Квадратик на лице
    f = cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), (0, 255, 255), 10)
    #if(timeToslip == 40):
    if(1):
    #Вычисления может быть исключение 
        try:
            face_descriptor2 = facerec.compute_face_descriptor(img, shape)
        except RuntimeError:
            pass
        
    #Растояние
        a = distance.euclidean(face_descriptor1, face_descriptor2)
        timeToslip = 0
    #Текст под  
    f = cv2.putText(f, "mark -{}".format(a), (d.left(), d.bottom()),cv2.FONT_HERSHEY_SIMPLEX, 1, (30, 105, 210), 2)
    #Вывод изображения
    cv2.imshow('mark or not', f)
    key = cv2.waitKey(100)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()